In [1]:
# Run this code cell to import relevant libraries

import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
hotels = pd.read_csv("hotels.csv")
hotels = hotels.fillna(0)
hotels = hotels.drop(columns = ["company","reservation_status","reservation_status_date"])
print(hotels.head)

<bound method NDFrame.head of                hotel  is_canceled  lead_time  arrival_date_year  \
0       Resort Hotel            0        342               2015   
1       Resort Hotel            0        737               2015   
2       Resort Hotel            0          7               2015   
3       Resort Hotel            0         13               2015   
4       Resort Hotel            0         14               2015   
...              ...          ...        ...                ...   
119385    City Hotel            0         23               2017   
119386    City Hotel            0        102               2017   
119387    City Hotel            0         34               2017   
119388    City Hotel            0        109               2017   
119389    City Hotel            0        205               2017   

       arrival_date_month  arrival_date_week_number  \
0                    July                        27   
1                    July                        27   


In [3]:
from sklearn.preprocessing import OneHotEncoder

target = hotels["adr"]
tot = hotels.drop(columns = ["adr"])
cats = hotels[["hotel","arrival_date_month", "customer_type", "meal", "country", "market_segment","distribution_channel", "reserved_room_type","assigned_room_type", "deposit_type"]].astype(str).values
categorical = OneHotEncoder().fit_transform(cats).toarray()
numerical = tot.drop(columns = ["hotel","arrival_date_month", "customer_type", "meal", "country", "market_segment","distribution_channel", "reserved_room_type","assigned_room_type", "deposit_type"]).values
data = np.append(categorical, numerical, axis=1)
print(data)
print(categorical)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 4.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 2.]]
[[0. 1. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]]


In [4]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=0.3, random_state=216)

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


linear_model = LinearRegression()
linear_model.fit(X=data, y=target)
predicted = linear_model.predict(data)

mse = mean_squared_error(target, predicted)
r2 = r2_score(target, predicted)
print(mse)
print(r2)

1127.6136952033266
0.5584642671089128


In [6]:
data = pd.read_csv("hotels.csv")
data.drop(inplace=True, axis=1, labels=['agent', 'company','hotel','reservation_status_date'])
data.fillna(data.mode().iloc[0], inplace=True)

In [7]:
X = data.iloc[:,1:]
y = data.iloc[:,0]

In [8]:
from sklearn.compose import make_column_transformer
ct = make_column_transformer(
    (OneHotEncoder(),['meal','distribution_channel','reservation_status','country','arrival_date_month','market_segment','deposit_type','customer_type', 'reserved_room_type','assigned_room_type' ]), remainder = 'passthrough'
    )
X = ct.fit_transform(X).toarray()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_

In [12]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, max_iter=1000)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [15]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[14904,    30],
       [   22,  8922]])

23849 / 23878 * 100 = 99.85% accuracy